Downloads the net balancing volume in weekly chunks (max api will allow), combines the weekly chunks, creates a new column (1 for positive balance request, 0 for negative balance request), outputs a csv titled balancing_volume_2020_2024.csv

In [ ]:
import pandas as pd 
import requests 
from datetime import datetime, timedelta
import plotly.graph_objects as go
# !pip install nbformat 

In [ ]:
start_date = datetime(2020, 1, 1)
end_date = datetime(2024, 12, 31)
dates = []

current_date = start_date
while current_date <= end_date:
    dates.append(current_date.strftime('%Y-%m-%d'))
    current_date += timedelta(days=7)

print(dates)  # Or use dates list as needed

In [9]:
dfs = []

for i in range(len(dates)-1):
    url = f'https://data.elexon.co.uk/bmrs/api/v1/balancing/nonbm/disbsad/summary?from={dates[i]}T00%3A00Z&to={dates[i+1]}T00%3A00Z'
    response = requests.get(url).json()
    df = pd.DataFrame(response['data'])
    dfs.append(df)

complete_df = pd.concat(dfs, ignore_index=True)
complete_df = complete_df.sort_values(by = 'startTime')

C:\Users\tommo\AppData\Local\Temp\ipykernel_17628\3199940851.py:9: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



In [10]:
fig = go.Figure(go.Scatter(x = complete_df['startTime'], y = complete_df['netVolume']))
fig.show()

In [11]:
output_df = complete_df[['settlementDate', 'settlementPeriod', 'startTime', 'netVolume']]
output_df['isPositive'] = (output_df['netVolume'] > 0).astype(int)
output_df

C:\Users\tommo\AppData\Local\Temp\ipykernel_17628\4135766150.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,settlementDate,settlementPeriod,startTime,netVolume,isPositive
201,2020-01-01,1,2020-01-01T00:00:00Z,-337.500,0
200,2020-01-01,2,2020-01-01T00:30:00Z,-337.500,0
199,2020-01-01,3,2020-01-01T01:00:00Z,-337.500,0
198,2020-01-01,4,2020-01-01T01:30:00Z,-337.500,0
197,2020-01-01,5,2020-01-01T02:00:00Z,-337.500,0
...,...,...,...,...,...
62697,2024-12-24,45,2024-12-24T22:00:00Z,33.933,1
62696,2024-12-24,46,2024-12-24T22:30:00Z,47.440,1
62695,2024-12-24,47,2024-12-24T23:00:00Z,192.450,1
62694,2024-12-24,48,2024-12-24T23:30:00Z,109.500,1


In [12]:
output_df.to_csv('balancing_volume_2020_2024.csv')

Creating a date, settlement period, time csv to use for conversion of other data formats. 

In [13]:
setttlement_period_lookup_file = output_df[['settlementDate', 'settlementPeriod', 'startTime']]
setttlement_period_lookup_file.to_csv('time_to_settlement_period_lookup_table.csv')